In [2]:
# scrapy 
# 비동기식 방식으로 빠름 ,

In [1]:
import scrapy, requests 
from scrapy.http import TextResponse

In [3]:
#  g마켓 베스트 200 r
#  1.  스크래피 프로젝트생성


In [8]:
!scrapy startproject gmarket

New Scrapy project 'gmarket', using template directory 'c:\users\user\anaconda3\lib\site-packages\scrapy\templates\project', created in:
    c:\Users\User\Desktop\20220803크롤링\day3\notebooks\gmarket

You can start your first spider with:
    cd gmarket
    scrapy genspider example example.com


In [9]:
!tree gmarket /f

���� PATH�� ����Դϴ�.
���� �Ϸ� ��ȣ�� 000000AA 7C6C:FC35�Դϴ�.
C:\USERS\USER\DESKTOP\20220803ũ�Ѹ�\DAY3\NOTEBOOKS\GMARKET
��  scrapy.cfg
��  
����gmarket
    ��  items.py
    ��  middlewares.py
    ��  pipelines.py
    ��  settings.py
    ��  __init__.py
    ��  
    ����spiders
            __init__.py
            


In [ ]:
# items.py 수집할데이터의 구조정의 
#  middlewares.py  ㅔㄷ이터를 수집할떄 headers 정보와 같은내용을 설정
# pipelines.py  데이터를 수집한 후에 코들를 실행정의 
# settings.py 크롤링 설정 : 크롤링 시간 텀, robots.txt 규칙 
# spiders : 디렉토리 : 크롤링 절차 정의,

In [10]:
#  2. xpath 찾기 : 링크 , 상세페이지 데이터 
url = 'http://corners.gmarket.co.kr/Bestsellers'

In [11]:
# f링틐 데이터 수집
request = requests.get('http://corners.gmarket.co.kr/Bestsellers')
response = TextResponse(request.url , body = request.text ,encoding='utf-8')

In [14]:
links= response.xpath('//*[@id="gBestWrap"]/div/div[3]/div/ul/li/a/@href').extract()
links[:2]

['http://item.gmarket.co.kr/Item?goodscode=2519604129&ver=637953125829377467',
 'http://item.gmarket.co.kr/Item?goodscode=2520466046&ver=637953125829377467']

In [15]:
# 상세페이지 데이터수집
link = links[0]
request = requests.get(link)
response = TextResponse(request.url, body=request.text, encoding='utf-8')

In [20]:
title = response.xpath('//*[@id="itemcase_basic"]/div/h1/text()')[0].extract()
price = response.xpath('//*[@id="itemcase_basic"]/div/p/span/strong/text()')[0].extract()
title,price

('브라운 면도기 세정액 CCR5+1 ', '25,030')

In [5]:
# 3 .item.py : 코드 작성 : model(데이터의 구조)

In [21]:
%%writefile gmarket/gmarket/items.py
import scrapy 

class GmarketItem(scrapy.Item):
    title = scrapy.Field()
    price = scrapy.Field()
    link = scrapy.Field()

Overwriting gmarket/gmarket/items.py


In [ ]:
# 4. spider.py 코드작성: 크롤링 절차정의

In [23]:
%%writefile gmarket/gmarket/spiders/spider.py
import scrapy
from gmarket.items import GmarketItem

class GMSpider(scrapy.Spider):
    name = "GMB"
    allow_domain = ["gmarket.co.kr"]
    start_urls = ["http://corners.gmarket.co.kr/Bestsellers"]
    
    def parse(self, response):
        links = response.xpath('//*[@id="gBestWrap"]/div/div[3]/div/ul/li/a/@href').extract()
        for link in links[:20]:
            yield scrapy.Request(link, callback=self.parse_content)
    
    def parse_content(self, response):
        item = GmarketItem()
        item["title"] = response.xpath('//*[@id="itemcase_basic"]/div[1]/h1/text()')[0].extract()
        item["price"] = response.xpath('//*[@id="itemcase_basic"]/div[1]/p/span/strong/text()')[0].extract()
        item["link"] = response.url
        yield item

Overwriting gmarket/gmarket/spiders/spider.py


In [7]:
# 5. scrapy 프로젝트 실행

In [24]:
%pwd

'c:\\Users\\User\\Desktop\\20220803크롤링\\day3\\notebooks'

In [ ]:
cd 'c:\\Users\\User\\Desktop\\20220803크롤링\\day3\\notebooks'

In [ ]:
scrapy crawl GMB -o items.csv

In [26]:
pd.read_csv('gmarket/items.csv')[:2]

,link,price,title
0,http://item.gmarket.co.kr/Item?goodscode=25149...,"7,900",DCOVERS/디커버스/반팔티/티셔츠/여성/남자/빅사이즈
1,http://item.gmarket.co.kr/Item?goodscode=25196...,"25,030",브라운 면도기 세정액 CCR5+1
